In [1]:
pip install torch torchvision transformers numpy matplotlib

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ------------- -------------------------- 3.4/10.2 MB 20.2 MB/s eta 0:00:01
   -------------------------------- ------- 8.4/10.2 MB 23.7 MB/s eta 0:00:01
   ---------------------------------------- 10.2/10.2 MB 22.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 34.5 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt


c:\Users\jayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

c:\Users\jayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jayan\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 766604.27 examples/s]


In [8]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [10]:
print (train_dataset)


Dataset({
    features: ['text'],
    num_rows: 36718
})


In [11]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Function to count tokens in a dataset
def count_tokens(dataset):
    total_tokens = 0
    for example in dataset:
        tokens = tokenizer.tokenize(example['text'])
        total_tokens += len(tokens)
    return total_tokens

# Count tokens for each dataset
train_tokens = count_tokens(train_dataset)
val_tokens = count_tokens(val_dataset)
test_tokens = count_tokens(test_dataset)

print(f"Number of tokens in train dataset: {train_tokens}")
print(f"Number of tokens in validation dataset: {val_tokens}")
print(f"Number of tokens in test dataset: {test_tokens}")

c:\Users\jayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jayan\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Token indices sequence length is longer than the specified maximum sequence length for this model

Number of tokens in train dataset: 2303695
Number of tokens in validation dataset: 238656
Number of tokens in test dataset: 273178


In [12]:
from collections import Counter
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Function to tokenize text and build vocabulary
def build_vocabulary(dataset, tokenizer, vocab_size=10000, min_freq=5):
    token_counter = Counter()
    
    # Tokenize the dataset and count token frequencies
    for example in dataset:
        tokens = tokenizer.tokenize(example['text'])
        token_counter.update(tokens)
    
    # Filter out rare tokens
    vocab = [token for token, freq in token_counter.items() if freq >= min_freq]
    
    # Keep the top `vocab_size` most frequent tokens
    vocab = vocab[:vocab_size]
    
    # Add special tokens
    special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
    vocab = special_tokens + vocab
    
    # Create token to index mapping
    token_to_idx = {token: idx for idx, token in enumerate(vocab)}
    
    return token_to_idx

# Build vocabulary for the training dataset
vocab = build_vocabulary(train_dataset, tokenizer)

# Function to convert tokens to indices
def tokens_to_indices(tokens, token_to_idx):
    return [token_to_idx.get(token, token_to_idx["<UNK>"]) for token in tokens]

# Function to create training examples
def create_training_examples(dataset, tokenizer, token_to_idx, seq_len=8):
    inputs = []
    targets = []
    
    for example in dataset:
        tokens = tokenizer.tokenize(example['text'])
        token_indices = tokens_to_indices(tokens, token_to_idx)
        
        for i in range(len(token_indices) - seq_len):
            inputs.append(token_indices[i:i+seq_len])
            targets.append(token_indices[i+seq_len])
    
    return inputs, targets

# Create training examples for the training dataset
train_inputs, train_targets = create_training_examples(train_dataset, tokenizer, vocab)

# Function to split data into batches
def create_batches(inputs, targets, batch_size=64):
    batches = []
    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]
        batch_targets = targets[i:i+batch_size]
        batches.append((batch_inputs, batch_targets))
    return batches

# Create batches for the training dataset
train_batches = create_batches(train_inputs, train_targets)

Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
